### Using AWS SageMaker for Inference
On AWS sagemaker, using a pretrained Object Detection Model works best with tensorflow 2.8 and needs an updated version of sagemaker to use this version of tensorflow.

In [54]:
!pip install tensorflow_hub
%pip install -U sagemaker
!pip install tensorflow==2.8

You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


''

Import the necessary libraries

In [55]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub
import sagemaker
import json
import cv2
tf.get_logger().setLevel('ERROR')

In [56]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
dataset_bucket = "swifty-datasets"

bucket = sess.default_bucket()

print("Bucket: {}".format(bucket))
print("SageMaker ver: " + sagemaker.__version__)
print("Tensorflow ver: " + tf.__version__)

Bucket: sagemaker-us-east-1-038469568353
SageMaker ver: 2.88.2
Tensorflow ver: 2.8.0


In [57]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data='s3://swifty-ai-models/other_models/object_detector.tar.gz', role=role,framework_version = '2.8')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----!

In [113]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('https')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data)) 
  basewidth = 800 #Resize to basewidth of 800 and resize it proportionally to the height
  wpercent = (basewidth/float(image.size[0]))
  hsize = int((float(image.size[1])*float(wpercent)))
  image = image.resize((basewidth,hsize), Image.ANTIALIAS)
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)

In [114]:
image_path = 'https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg' 

In [115]:
input_tensor = load_image_into_numpy_array(image_path)

In [116]:
input = {
  'instances': input_tensor.tolist()
}

In [117]:
#print(input_tensor.tolist())

In [118]:
result = predictor.predict(input)

In [155]:
detection_scores= result['predictions'][0]['detection_scores']
detection_classes= result['predictions'][0]['detection_classes']

In [120]:
print(result['predictions'][0]['detection_classes'])

[16.0, 84.0, 86.0, 86.0, 64.0, 72.0, 16.0, 88.0, 16.0, 16.0, 5.0, 16.0, 65.0, 16.0, 84.0, 82.0, 16.0, 85.0, 1.0, 1.0, 16.0, 17.0, 64.0, 88.0, 16.0, 86.0, 19.0, 86.0, 38.0, 84.0, 16.0, 1.0, 16.0, 16.0, 51.0, 64.0, 16.0, 61.0, 16.0, 84.0, 16.0, 3.0, 16.0, 42.0, 16.0, 84.0, 50.0, 16.0, 51.0, 16.0, 48.0, 3.0, 16.0, 1.0, 44.0, 16.0, 24.0, 1.0, 16.0, 51.0, 16.0, 28.0, 88.0, 61.0, 84.0, 28.0, 34.0, 64.0, 48.0, 1.0, 62.0, 84.0, 73.0, 16.0, 64.0, 24.0, 1.0, 17.0, 5.0, 64.0, 16.0, 50.0, 85.0, 84.0, 16.0, 64.0, 86.0, 64.0, 67.0, 87.0, 43.0, 64.0, 1.0, 28.0, 16.0, 44.0, 16.0, 8.0, 84.0, 1.0]


In [105]:
def unique(list1):
     
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    print(unique_list)

In [106]:
unique(result['predictions'][0]['detection_classes'])

[1.0, 3.0, 5.0, 8.0, 16.0, 17.0, 19.0, 24.0, 28.0, 34.0, 38.0, 42.0, 43.0, 44.0, 48.0, 50.0, 51.0, 61.0, 62.0, 64.0, 65.0, 67.0, 72.0, 73.0, 82.0, 84.0, 85.0, 86.0, 87.0, 88.0]


In [145]:
print(len((result['predictions'][0]['detection_scores'])))
print(len((result['predictions'][0]['detection_classes'])))

100
100


In [146]:
txt_file = open("cococlasses.txt", "r")
content = txt_file.read().split("\n")
print(content)

['background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


In [139]:
print(len(content))

92


In [141]:
print(content[16])

bird


In [150]:
def convertToInt(list1):
    detected_classes=[]
    for i in range(len(list1)):
        detected_classes.append(int(list1[i]))
    return detected_classes 

In [151]:
detected_classes = convertToInt(result['predictions'][0]['detection_classes'])

In [159]:
def findTop10Classes(list1, list2, content):
    print("The system has detected the following objects:")
    print("Object \t\t\t Confidence")
    for i in range(10):
        print(content[list1[i]].capitalize(),"\t\t\t",list2[i]*100)

In [160]:
findTop10Classes(detected_classes,detection_scores,content)

The system has detected the following objects:
Object 			 Confidence
Bird 			 64.2007589
Book 			 48.472
Vase 			 46.1754799
Vase 			 45.5887854
Potted plant 			 44.4863498
Tv 			 43.046379099999996
Bird 			 40.3677404
Teddy bear 			 39.5147175
Bird 			 37.4859095
Bird 			 35.2080941


### Delete End Point

In [161]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Without using AWS services

In [ ]:
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [ ]:
image_path = 'https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg'
image_np = load_image_into_numpy_array(image_path)

plt.figure(figsize=(24,32))
plt.imshow(image_np[0])
plt.show()


In [ ]:
# running inference
results = detector(image_np)
 #needs tf 2.8 to work

result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

In [ ]:
print(image_np)

In [ ]:
print(result['detection_scores'])

In [ ]:
print(result['detection_classes'])
#Get image,
#Predict and return output from end point
#After predicting, Find way to add detection boxes to output and then save to AWS s3 on the image.